<a href="https://colab.research.google.com/github/eekaiboon/gen_ai/blob/main/transaction_categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal

Automate expense categorization using LLM

#Setup

*   Use OpenAI Assistant API to build this AI agent
*   Provide external tools such as access to database (e.g. table that stores accounting fields) for the OpenAI assistant

In [29]:
!pip install --upgrade llama-index

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.6/861.6 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: llama-index
    Found existing installation: llama-index 0.8.67
    Uninstalling llama-index-0.8.67:
      Successfully uninstalled llama-index-0.8.67


# Create Database Schema

In [47]:
import enum

from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    Enum,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

# Create accounting fields table
class FieldType(enum.Enum):
    general_ledger_account = 1
    location = 2

table_name = "accounting_fields"
accounting_fields_table = Table(
    table_name,
    metadata_obj,
    Column("accounting_field_id", String(128), primary_key=True),
    Column("accounting_field_type", Enum(FieldType)),
    Column("accounting_field_value", String(128)),
    Column("customer_account_id", String(128)),
)
metadata_obj.create_all(engine)

# Define SQL Database

In [48]:
from llama_index import SQLDatabase, ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model='gpt-4-1106-preview')
service_context = ServiceContext.from_defaults(llm=llm)

sql_database = SQLDatabase(engine, include_tables=[table_name])

In [69]:
from sqlalchemy import insert, delete

# Clear the table first.
stmt = delete(accounting_fields_table)
with engine.begin() as connection:
  cursor = connection.execute(stmt)

# Insert rows.
rows = [
    {"accounting_field_id": "f_1", "accounting_field_type": FieldType.general_ledger_account, "accounting_field_value": "Office Equipment", "customer_account_id": "cuacc_1"},
    {"accounting_field_id": "f_2", "accounting_field_type": FieldType.general_ledger_account, "accounting_field_value": "Cost of Goods Sold", "customer_account_id": "cuacc_1"},
    {"accounting_field_id": "f_3", "accounting_field_type": FieldType.general_ledger_account, "accounting_field_value": "Accounts Payable", "customer_account_id": "cuacc_1"},
    {"accounting_field_id": "f_4", "accounting_field_type": FieldType.general_ledger_account, "accounting_field_value": "Accounts Receivable", "customer_account_id": "cuacc_1"},
    {"accounting_field_id": "f_5", "accounting_field_type": FieldType.location, "accounting_field_value": "New York", "customer_account_id": "cuacc_1"},
    {"accounting_field_id": "f_6", "accounting_field_type": FieldType.location, "accounting_field_value": "San Francisco", "customer_account_id": "cuacc_1"},
]
for row in rows:
    stmt = insert(accounting_fields_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [70]:
# View current table.

stmt = select(
    accounting_fields_table.c.accounting_field_id,
    accounting_fields_table.c.accounting_field_type,
    accounting_fields_table.c.accounting_field_value,
    accounting_fields_table.c.customer_account_id,
).select_from(accounting_fields_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('f_1', <FieldType.general_ledger_account: 1>, 'Office Equipment', 'cuacc_1'), ('f_2', <FieldType.general_ledger_account: 1>, 'Cost of Goods Sold', 'cuacc_1'), ('f_3', <FieldType.general_ledger_account: 1>, 'Accounts Payable', 'cuacc_1'), ('f_4', <FieldType.general_ledger_account: 1>, 'Accounts Receivable', 'cuacc_1'), ('f_5', <FieldType.location: 2>, 'New York', 'cuacc_1'), ('f_6', <FieldType.location: 2>, 'San Francisco', 'cuacc_1')]


# Define Text-to-SQL Query Engine Tool

In [71]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["accounting_fields"],
)
query_str = "What are the general ledger accounts of `cuacc_1`"
response = query_engine.query(query_str)

In [72]:
from IPython.display import Markdown, display

display(Markdown(f"<b>{response}</b>"))

<b>The general ledger accounts of `cuacc_1` are Office Equipment, Cost of Goods Sold, Accounts Payable, and Accounts Receivable.</b>

In [73]:
from llama_index.tools.query_engine import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="sql_tool",
    description=(
        """Useful for translating a natural language query into a SQL query over
           a table named `accounting_fields`, which contains accounting_field_id,
           accounting_field_type, accounting_field_value, and customer_accounting_id.
        """
    ),
)

# Initialize Agent

In [74]:
from llama_index.agent import OpenAIAssistantAgent

agent = OpenAIAssistantAgent.from_new(
    name="Accountant bot",
    instructions="""You are a bot designed to perform accounting related job,
      just like an accountant. Specifically, you are capable of understanding
      expenses by employees and assign accounting fields to them so that we can
      import the expenses with correct categorization of accounting fields
      before importing them to accounting software.
    """,
    tools=[sql_tool],
    verbose=True,
)

In [75]:
expense = """
{
  "expense_id": "exp_1",
  "amount_usd_cents": 1000,
  "accrued_at_epoch": 1699750201,
  "memo": "Work laptop for new engineer based in New York",
  "customer_account_id": "cuacc_1"
}
"""

user_query = f"""
You are given the following expense in json.

Expense:
```json
{expense}
```

Please figure out how we should set the general ledger account accounting field
for the expense.
""".replace(
    "{", "{{"
).replace(
    "}", "}}"
)

print(user_query)


You are given the following expense in json.

Expense:
```json

{{
  "expense_id": "exp_1",
  "amount_usd_cents": 1000,
  "accrued_at_epoch": 1699750201,
  "memo": "Work laptop for new engineer based in New York",
  "customer_account_id": "cuacc_1"
}}

```

Please figure out how we should set the general ledger account accounting field
for the expense.



In [76]:
response = agent.chat(user_query)
print(str(response))

=== Calling Function ===
Calling function: sql_tool with args: {
  "input": "SELECT * FROM accounting_fields WHERE accounting_field_type = 'general_ledger_account' AND customer_accounting_id = 'cuacc_1'"
}
Got output: The general ledger accounts for customer cuacc_1 are Office Equipment, Cost of Goods Sold, Accounts Payable, and Accounts Receivable.
To set the general ledger account accounting field for the expense related to the "Work laptop for new engineer based in New York," we should assign it to the "Office Equipment" general ledger account, as this expense is for an asset that the company will use over time. 

Thus, the general ledger account accounting field for expense_id "exp_1" will be "Office Equipment."
